In [6]:
team_name_to_abbr = {
    "Chi. Cubs": "CHC",
    "Pittsburgh": "PIT",
    "Miami": "MIA",
    "Baltimore": "BAL",
    "Philadelphia": "PHI",
    "Toronto": "TOR",
    "Cleveland": "CLE",
    "Boston": "BOS",
    "Milwaukee": "MIL",
    "Kansas City": "KCR",
    "Houston": "HOU",
    "LA Angels": "LAA",
    "San Francisco": "SFG",
    "NY Yankees": "NYY",
    "Colorado": "COL",
    "Cincinnati": "CIN",
    "Washington": "WSN",
    "Atlanta": "ATL",
    "Minnesota": "MIN",
    "St. Louis": "STL",
    "San Diego": "SDP",
    "Seattle": "SEA",
    "Arizona": "ARI",
    "Tampa Bay": "TBR",
    "Chi. White Sox": "CHW",
    "Detroit": "DET",
    "Oakland": "OAK",
    "NY Mets": "NYM",
    "Texas": "TEX",
    "LA Dodgers": "LAD",
    "Athletics": "OAK"  # in case some entries use "Athletics"
}


In [1]:
import os
import pandas as pd

# Path to folder with CSVs
folder = "daily_odds"

# Store all DataFrames
dfs = []

# Loop through each CSV in the folder
for file in sorted(os.listdir(folder)):
    if file.endswith(".csv"):
        try:
            path = os.path.join(folder, file)
            df = pd.read_csv(path)
            dfs.append(df)
        except Exception as e:
            print(f"❌ Failed to read {file}: {e}")

# Combine all into one DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Save to disk
combined_df.to_csv("all_daily_odds.csv", index=False)
print("✅ Saved to all_daily_odds.csv")


✅ Saved to all_daily_odds.csv


In [7]:
master = pd.read_csv('all_daily_odds.csv')
master['homeTeam_abbr'] = master['homeTeam'].map(team_name_to_abbr)
master['awayTeam_abbr'] = master['awayTeam'].map(team_name_to_abbr)
master

,gameId,startDate,homeTeam,awayTeam,homePitcher,awayPitcher,homeScore,awayScore,betmgm_opening_homeOdds,betmgm_opening_awayOdds,...,bet365_opening_homeOdds,bet365_opening_awayOdds,bet365_current_homeOdds,bet365_current_awayOdds,draftkings_opening_homeOdds,draftkings_opening_awayOdds,draftkings_current_homeOdds,draftkings_current_awayOdds,homeTeam_abbr,awayTeam_abbr
0,259012,2022-05-20T18:20:00+00:00,Chi. Cubs,Arizona,Kyle Hendricks,Humberto Castellanos,6,10,-135.0,110.0,...,-130.0,110.0,-145.0,125.0,-140.0,120.0,-140.0,120.0,CHC,ARI
1,259005,2022-05-20T22:35:00+00:00,Pittsburgh,St. Louis,Zach Thompson,Adam Wainwright,3,5,125.0,-155.0,...,125.0,-145.0,125.0,-145.0,115.0,-135.0,110.0,-140.0,PIT,STL
2,259013,2022-05-20T22:40:00+00:00,Miami,Atlanta,Trevor Rogers,Charlie Morton,3,5,105.0,-130.0,...,110.0,-130.0,110.0,-130.0,125.0,-145.0,-110.0,-120.0,MIA,ATL
3,259006,2022-05-20T23:05:00+00:00,Baltimore,Tampa Bay,Tyler Wells,Jalen Beeks,8,6,120.0,-145.0,...,135.0,-155.0,120.0,-140.0,110.0,-130.0,115.0,-135.0,BAL,TBR
4,259010,2022-05-20T23:05:00+00:00,Philadelphia,LA Dodgers,Ranger Suarez,Julio Urias,1,4,120.0,-145.0,...,125.0,-145.0,125.0,-145.0,125.0,-145.0,110.0,-140.0,PHI,LAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3612,301022,2024-08-21T00:10:00+00:00,Houston,Boston,Ronel Blanco,Nick Pivetta,5,6,-135.0,110.0,...,-130.0,110.0,-115.0,-105.0,-130.0,110.0,-110.0,-110.0,HOU,BOS
3613,301027,2024-08-21T01:40:00+00:00,Athletics,Tampa Bay,Joey Estes,Shane Baz,0,1,100.0,-120.0,...,100.0,-120.0,100.0,-120.0,-102.0,-118.0,102.0,-122.0,OAK,TBR
3614,301034,2024-08-21T01:40:00+00:00,San Diego,Minnesota,Martin Perez,Bailey Ober,7,5,100.0,-120.0,...,100.0,-120.0,105.0,-125.0,102.0,-122.0,105.0,-125.0,SDP,MIN
3615,301025,2024-08-21T01:45:00+00:00,San Francisco,Chi. White Sox,Robbie Ray,Davis Martin,4,1,-250.0,200.0,...,-230.0,190.0,-250.0,205.0,-238.0,195.0,-258.0,210.0,SFG,CHW


In [ ]:
import os
import pandas as pd
from datetime import datetime

# Load odds data
df_odds = master

# Create helper columns
df_odds['year'] = pd.to_datetime(df_odds['startDate']).dt.year
df_odds['game_date'] = pd.to_datetime(df_odds['startDate']).dt.strftime("%Y-%m-%d")

# Initialize empty columns
df_odds["homeTeam_win_pct"] = None
df_odds["awayTeam_win_pct"] = None

def get_previous_win_pct(team_abbr, opponent_abbr, date_str, year):
    file_path = f"season_win_percentage/{year}_{team_abbr}.csv"
    if not os.path.exists(file_path):
        return None
    df_team = pd.read_csv(file_path)
    df_team = df_team.sort_values("date")
    
    # Get the row where the team played that opponent on that date
    idx = df_team[
        (df_team["date"] == date_str) &
        (df_team["team_ID"] == team_abbr) &
        (df_team["opponent"] == opponent_abbr)
    ].index

    if len(idx) == 0 or idx[0] == 0:
        return None  # No match or no previous game
    prev_row = df_team.iloc[idx[0] - 1]
    return prev_row["win_percentage"]

# Loop through each game
for i, row in df_odds.iterrows():
    year = row["year"]
    date = row["game_date"]
    home = row["homeTeam_abbr"]
    away = row["awayTeam_abbr"]

    # Get win percentage from previous game
    home_pct = get_previous_win_pct(home, away, date, year)
    away_pct = get_previous_win_pct(away, home, date, year)

    df_odds.at[i, "homeTeam_win_pct"] = home_pct
    df_odds.at[i, "awayTeam_win_pct"] = away_pct

# Save updated DataFrame
df_odds

,gameId,startDate,homeTeam,awayTeam,homePitcher,awayPitcher,homeScore,awayScore,betmgm_opening_homeOdds,betmgm_opening_awayOdds,...,draftkings_opening_homeOdds,draftkings_opening_awayOdds,draftkings_current_homeOdds,draftkings_current_awayOdds,homeTeam_abbr,awayTeam_abbr,year,game_date,homeTeam_win_pct,awayTeam_win_pct
0,259012,2022-05-20T18:20:00+00:00,Chi. Cubs,Arizona,Kyle Hendricks,Humberto Castellanos,6,10,-135.0,110.0,...,-140.0,120.0,-140.0,120.0,CHC,ARI,2022,2022-05-20,0.405,0.475
1,259005,2022-05-20T22:35:00+00:00,Pittsburgh,St. Louis,Zach Thompson,Adam Wainwright,3,5,125.0,-155.0,...,115.0,-135.0,110.0,-140.0,PIT,STL,2022,2022-05-20,0.432,0.526
2,259013,2022-05-20T22:40:00+00:00,Miami,Atlanta,Trevor Rogers,Charlie Morton,3,5,105.0,-130.0,...,125.0,-145.0,-110.0,-120.0,MIA,ATL,2022,2022-05-20,0.459,0.447
3,259006,2022-05-20T23:05:00+00:00,Baltimore,Tampa Bay,Tyler Wells,Jalen Beeks,8,6,120.0,-145.0,...,110.0,-130.0,115.0,-135.0,BAL,TBR,2022,2022-05-20,0.385,0.605
4,259010,2022-05-20T23:05:00+00:00,Philadelphia,LA Dodgers,Ranger Suarez,Julio Urias,1,4,120.0,-145.0,...,125.0,-145.0,110.0,-140.0,PHI,LAD,2022,2022-05-20,0.474,0.676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3612,301022,2024-08-21T00:10:00+00:00,Houston,Boston,Ronel Blanco,Nick Pivetta,5,6,-135.0,110.0,...,-130.0,110.0,-110.0,-110.0,HOU,BOS,2024,2024-08-21,0.544,0.524
3613,301027,2024-08-21T01:40:00+00:00,Athletics,Tampa Bay,Joey Estes,Shane Baz,0,1,100.0,-120.0,...,-102.0,-118.0,102.0,-122.0,OAK,TBR,2024,2024-08-21,0.429,0.504
3614,301034,2024-08-21T01:40:00+00:00,San Diego,Minnesota,Martin Perez,Bailey Ober,7,5,100.0,-120.0,...,102.0,-122.0,105.0,-125.0,SDP,MIN,2024,2024-08-21,0.567,0.556
3615,301025,2024-08-21T01:45:00+00:00,San Francisco,Chi. White Sox,Robbie Ray,Davis Martin,4,1,-250.0,200.0,...,-238.0,195.0,-258.0,210.0,SFG,CHW,2024,2024-08-21,0.508,0.236


In [13]:
import os
import pandas as pd
from datetime import datetime, timedelta

# Add empty columns for batting average
df_odds["homeTeam_batting_avg"] = None
df_odds["awayTeam_batting_avg"] = None

def get_batting_average(team_abbr, date_str):
    batting_file = os.path.join("batting_data", f"{date_str}.csv")
    if not os.path.exists(batting_file):
        return None
    df_bat = pd.read_csv(batting_file)
    row = df_bat[df_bat["Team Abbreviation"] == team_abbr]
    if row.empty:
        return None
    return float(row.iloc[0]["Batting Average"])

# Process each row
for i, row in df_odds.iterrows():
    try:
        game_date = pd.to_datetime(row["game_date"])
        next_day = (game_date - timedelta(days=1)).strftime("%Y-%m-%d")

        home_abbr = row["homeTeam_abbr"]
        away_abbr = row["awayTeam_abbr"]

        home_avg = get_batting_average(home_abbr, next_day)
        away_avg = get_batting_average(away_abbr, next_day)

        df_odds.at[i, "homeTeam_batting_avg"] = home_avg
        df_odds.at[i, "awayTeam_batting_avg"] = away_avg
    except Exception as e:
        print(f"⚠️ Error on row {i}: {e}")

df_odds = df_odds.dropna(subset=["awayTeam_batting_avg"])
df_odds

,gameId,startDate,homeTeam,awayTeam,homePitcher,awayPitcher,homeScore,awayScore,betmgm_opening_homeOdds,betmgm_opening_awayOdds,...,draftkings_current_homeOdds,draftkings_current_awayOdds,homeTeam_abbr,awayTeam_abbr,year,game_date,homeTeam_win_pct,awayTeam_win_pct,homeTeam_batting_avg,awayTeam_batting_avg
8,259014,2022-05-21T00:10:00+00:00,Milwaukee,Washington,Eric Lauer,Erick Fedde,7,0,-190.0,155.0,...,-245.0,185.0,MIL,WSN,2022,2022-05-21,0.641,0.325,0.23328,0.249616
9,259009,2022-05-21T00:10:00+00:00,Kansas City,Minnesota,Daniel Lynch IV,Devin Smeltzer,4,6,105.0,-125.0,...,110.0,-130.0,KCR,MIN,2022,2022-05-21,0.368,0.59,0.230707,0.241129
10,259007,2022-05-21T00:10:00+00:00,Houston,Texas,Cristian Javier,Martin Perez,0,3,-185.0,150.0,...,-165.0,145.0,HOU,TEX,2022,2022-05-21,0.625,0.474,0.231481,0.221405
11,259002,2022-05-21T01:38:00+00:00,LA Angels,Oakland,Chase Gockel,Paul Blackburn,2,4,-185.0,150.0,...,-235.0,180.0,LAA,OAK,2022,2022-05-21,0.585,0.415,0.246257,0.205469
12,259004,2022-05-21T02:15:00+00:00,San Francisco,San Diego,Jakob Junis,Sean Manaea,7,8,-115.0,-105.0,...,-140.0,110.0,SFG,SDP,2022,2022-05-21,0.579,0.641,0.247573,0.227092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3612,301022,2024-08-21T00:10:00+00:00,Houston,Boston,Ronel Blanco,Nick Pivetta,5,6,-135.0,110.0,...,-110.0,-110.0,HOU,BOS,2024,2024-08-21,0.544,0.524,0.261227,0.261592
3613,301027,2024-08-21T01:40:00+00:00,Athletics,Tampa Bay,Joey Estes,Shane Baz,0,1,100.0,-120.0,...,102.0,-122.0,OAK,TBR,2024,2024-08-21,0.429,0.504,0.231528,0.231775
3614,301034,2024-08-21T01:40:00+00:00,San Diego,Minnesota,Martin Perez,Bailey Ober,7,5,100.0,-120.0,...,105.0,-125.0,SDP,MIN,2024,2024-08-21,0.567,0.556,0.26524,0.251004
3615,301025,2024-08-21T01:45:00+00:00,San Francisco,Chi. White Sox,Robbie Ray,Davis Martin,4,1,-250.0,200.0,...,-258.0,210.0,SFG,CHW,2024,2024-08-21,0.508,0.236,0.24318,0.219851


Match based on the data we have available for the pitchers

In [17]:
# Normalize pitcher names from gamelog for matching
valid_pitchers = set(p.strip().title() for p in df_pitchers['Player'].unique())

# Normalize home/away pitcher names in df_odds
df_odds['homePitcher_normalized'] = df_odds['homePitcher'].fillna('').str.strip().str.title()
df_odds['awayPitcher_normalized'] = df_odds['awayPitcher'].fillna('').str.strip().str.title()

# Filter rows where both home and away pitcher names exist in gamelog
df_odds_filtered = df_odds[
    df_odds['homePitcher_normalized'].isin(valid_pitchers) &
    df_odds['awayPitcher_normalized'].isin(valid_pitchers)
].copy()

print(f"✅ Filtered odds data: {len(df_odds_filtered)} rows (from original {len(df_odds)})")

✅ Filtered odds data: 2351 rows (from original 3587)


/var/folders/hn/7wzd6_fs6wncrcy89b9jdj100000gn/T/ipykernel_46107/4048303913.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odds['homePitcher_normalized'] = df_odds['homePitcher'].fillna('').str.strip().str.title()
/var/folders/hn/7wzd6_fs6wncrcy89b9jdj100000gn/T/ipykernel_46107/4048303913.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odds['awayPitcher_normalized'] = df_odds['awayPitcher'].fillna('').str.strip().str.title()


In [20]:
df_pitchers = pd.read_csv("combined_pitcher_gamelogs.csv")

# Normalize
df_odds_filtered['homePitcher_normalized'] = df_odds_filtered['homePitcher'].fillna('').str.strip().str.title()
df_odds_filtered['awayPitcher_normalized'] = df_odds_filtered['awayPitcher'].fillna('').str.strip().str.title()
df_odds_filtered['game_date'] = pd.to_datetime(df_odds_filtered['startDate']).dt.date
df_odds_filtered['game_year'] = pd.to_datetime(df_odds_filtered['startDate']).dt.year

# Ensure pitcher data is clean
df_pitchers['Player'] = df_pitchers['Player'].str.strip().str.title()
df_pitchers['Date'] = pd.to_datetime(df_pitchers['Date'] + ' ' + df_pitchers['Year'].astype(str))

# ERA lookup function
def get_latest_era(pitcher_name, game_date):
    games = df_pitchers[df_pitchers['Player'] == pitcher_name]
    games = games[games['Date'].dt.date < game_date]
    if games.empty:
        return None
    latest_game = games.sort_values('Date', ascending=False).iloc[0]
    return latest_game['ERA']

# Add ERA columns
df_odds_filtered['homePitcher_era'] = df_odds_filtered.apply(
    lambda row: get_latest_era(row['homePitcher_normalized'], row['game_date']), axis=1)

df_odds_filtered['awayPitcher_era'] = df_odds_filtered.apply(
    lambda row: get_latest_era(row['awayPitcher_normalized'], row['game_date']), axis=1)

df_odds_filtered

,gameId,startDate,homeTeam,awayTeam,homePitcher,awayPitcher,homeScore,awayScore,betmgm_opening_homeOdds,betmgm_opening_awayOdds,...,game_date,homeTeam_win_pct,awayTeam_win_pct,homeTeam_batting_avg,awayTeam_batting_avg,homePitcher_normalized,awayPitcher_normalized,game_year,homePitcher_era,awayPitcher_era
8,259014,2022-05-21T00:10:00+00:00,Milwaukee,Washington,Eric Lauer,Erick Fedde,7,0,-190.0,155.0,...,2022-05-21,0.641,0.325,0.23328,0.249616,Eric Lauer,Erick Fedde,2022,2.16,4.08
10,259007,2022-05-21T00:10:00+00:00,Houston,Texas,Cristian Javier,Martin Perez,0,3,-185.0,150.0,...,2022-05-21,0.625,0.474,0.231481,0.221405,Cristian Javier,Martin Perez,2022,2.87,1.64
13,259026,2022-05-21T17:05:00+00:00,NY Yankees,Chi. White Sox,Nestor Cortes,Dallas Keuchel,7,5,-225.0,180.0,...,2022-05-21,0.737,0.5,0.245433,0.231073,Nestor Cortes,Dallas Keuchel,2022,1.35,5.54
15,259023,2022-05-21T19:07:00+00:00,Toronto,Cincinnati,Alek Manoah,Hunter Greene,3,1,-250.0,200.0,...,2022-05-21,0.538,0.289,0.2343,0.218593,Alek Manoah,Hunter Greene,2022,1.71,6.21
16,265633,2022-05-21T19:10:00+00:00,Colorado,NY Mets,German Marquez,Carlos Carrasco,1,5,-105.0,-115.0,...,2022-05-21,0.486,0.65,0.263789,0.252246,German Marquez,Carlos Carrasco,2022,6.16,3.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3611,301021,2024-08-21T00:10:00+00:00,Kansas City,LA Angels,Cole Ragans,Tyler Anderson,5,9,-210.0,170.0,...,2024-08-21,0.556,0.429,0.256483,0.2343,Cole Ragans,Tyler Anderson,2024,5.92,3.40
3612,301022,2024-08-21T00:10:00+00:00,Houston,Boston,Ronel Blanco,Nick Pivetta,5,6,-135.0,110.0,...,2024-08-21,0.544,0.524,0.261227,0.261592,Ronel Blanco,Nick Pivetta,2024,3.14,4.61
3614,301034,2024-08-21T01:40:00+00:00,San Diego,Minnesota,Martin Perez,Bailey Ober,7,5,100.0,-120.0,...,2024-08-21,0.567,0.556,0.26524,0.251004,Martin Perez,Bailey Ober,2024,5.20,3.54
3615,301025,2024-08-21T01:45:00+00:00,San Francisco,Chi. White Sox,Robbie Ray,Davis Martin,4,1,-250.0,200.0,...,2024-08-21,0.508,0.236,0.24318,0.219851,Robbie Ray,Davis Martin,2024,3.71,3.22


In [29]:
import numpy as np
import pandas as pd

# Convert relevant columns to numeric
df_odds_filtered["homeTeam_win_pct"] = pd.to_numeric(df_odds_filtered["homeTeam_win_pct"], errors="coerce")
df_odds_filtered["awayTeam_win_pct"] = pd.to_numeric(df_odds_filtered["awayTeam_win_pct"], errors="coerce")
df_odds_filtered["homeTeam_batting_avg"] = pd.to_numeric(df_odds_filtered["homeTeam_batting_avg"], errors="coerce")
df_odds_filtered["awayTeam_batting_avg"] = pd.to_numeric(df_odds_filtered["awayTeam_batting_avg"], errors="coerce")
df_odds_filtered["homePitcher_era"] = pd.to_numeric(df_odds_filtered["homePitcher_era"], errors="coerce")
df_odds_filtered["awayPitcher_era"] = pd.to_numeric(df_odds_filtered["awayPitcher_era"], errors="coerce")

# Compute alpha, beta, gamma
df_odds_filtered["alpha"] = df_odds_filtered["homeTeam_win_pct"] / df_odds_filtered["awayTeam_win_pct"]
df_odds_filtered["beta"] = df_odds_filtered["homeTeam_batting_avg"] / df_odds_filtered["awayTeam_batting_avg"]
df_odds_filtered["gamma"] = df_odds_filtered["awayPitcher_era"] / df_odds_filtered["homePitcher_era"]

df_odds_filtered


,gameId,startDate,homeTeam,awayTeam,homePitcher,awayPitcher,homeScore,awayScore,betmgm_opening_homeOdds,betmgm_opening_awayOdds,...,homeTeam_batting_avg,awayTeam_batting_avg,homePitcher_normalized,awayPitcher_normalized,game_year,homePitcher_era,awayPitcher_era,alpha,beta,gamma
8,259014,2022-05-21T00:10:00+00:00,Milwaukee,Washington,Eric Lauer,Erick Fedde,7,0,-190.0,155.0,...,0.233280,0.249616,Eric Lauer,Erick Fedde,2022,2.16,4.08,1.972308,0.934555,1.888889
10,259007,2022-05-21T00:10:00+00:00,Houston,Texas,Cristian Javier,Martin Perez,0,3,-185.0,150.0,...,0.231481,0.221405,Cristian Javier,Martin Perez,2022,2.87,1.64,1.318565,1.045509,0.571429
13,259026,2022-05-21T17:05:00+00:00,NY Yankees,Chi. White Sox,Nestor Cortes,Dallas Keuchel,7,5,-225.0,180.0,...,0.245433,0.231073,Nestor Cortes,Dallas Keuchel,2022,1.35,5.54,1.474000,1.062145,4.103704
15,259023,2022-05-21T19:07:00+00:00,Toronto,Cincinnati,Alek Manoah,Hunter Greene,3,1,-250.0,200.0,...,0.234300,0.218593,Alek Manoah,Hunter Greene,2022,1.71,6.21,1.861592,1.071855,3.631579
16,265633,2022-05-21T19:10:00+00:00,Colorado,NY Mets,German Marquez,Carlos Carrasco,1,5,-105.0,-115.0,...,0.263789,0.252246,German Marquez,Carlos Carrasco,2022,6.16,3.73,0.747692,1.045761,0.605519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3611,301021,2024-08-21T00:10:00+00:00,Kansas City,LA Angels,Cole Ragans,Tyler Anderson,5,9,-210.0,170.0,...,0.256483,0.234300,Cole Ragans,Tyler Anderson,2024,5.92,3.40,1.296037,1.094678,0.574324
3612,301022,2024-08-21T00:10:00+00:00,Houston,Boston,Ronel Blanco,Nick Pivetta,5,6,-135.0,110.0,...,0.261227,0.261592,Ronel Blanco,Nick Pivetta,2024,3.14,4.61,1.038168,0.998605,1.468153
3614,301034,2024-08-21T01:40:00+00:00,San Diego,Minnesota,Martin Perez,Bailey Ober,7,5,100.0,-120.0,...,0.265240,0.251004,Martin Perez,Bailey Ober,2024,5.20,3.54,1.019784,1.056716,0.680769
3615,301025,2024-08-21T01:45:00+00:00,San Francisco,Chi. White Sox,Robbie Ray,Davis Martin,4,1,-250.0,200.0,...,0.243180,0.219851,Robbie Ray,Davis Martin,2024,3.71,3.22,2.152542,1.106113,0.867925


In [30]:
# Ensure numeric
df_odds_filtered["homeScore"] = pd.to_numeric(df_odds_filtered["homeScore"], errors="coerce")
df_odds_filtered["awayScore"] = pd.to_numeric(df_odds_filtered["awayScore"], errors="coerce")

# Compute binary outcome
df_odds_filtered["X"] = (df_odds_filtered["homeScore"] > df_odds_filtered["awayScore"]).astype(int)
df_odds_filtered

,gameId,startDate,homeTeam,awayTeam,homePitcher,awayPitcher,homeScore,awayScore,betmgm_opening_homeOdds,betmgm_opening_awayOdds,...,awayTeam_batting_avg,homePitcher_normalized,awayPitcher_normalized,game_year,homePitcher_era,awayPitcher_era,alpha,beta,gamma,X
8,259014,2022-05-21T00:10:00+00:00,Milwaukee,Washington,Eric Lauer,Erick Fedde,7,0,-190.0,155.0,...,0.249616,Eric Lauer,Erick Fedde,2022,2.16,4.08,1.972308,0.934555,1.888889,1
10,259007,2022-05-21T00:10:00+00:00,Houston,Texas,Cristian Javier,Martin Perez,0,3,-185.0,150.0,...,0.221405,Cristian Javier,Martin Perez,2022,2.87,1.64,1.318565,1.045509,0.571429,0
13,259026,2022-05-21T17:05:00+00:00,NY Yankees,Chi. White Sox,Nestor Cortes,Dallas Keuchel,7,5,-225.0,180.0,...,0.231073,Nestor Cortes,Dallas Keuchel,2022,1.35,5.54,1.474000,1.062145,4.103704,1
15,259023,2022-05-21T19:07:00+00:00,Toronto,Cincinnati,Alek Manoah,Hunter Greene,3,1,-250.0,200.0,...,0.218593,Alek Manoah,Hunter Greene,2022,1.71,6.21,1.861592,1.071855,3.631579,1
16,265633,2022-05-21T19:10:00+00:00,Colorado,NY Mets,German Marquez,Carlos Carrasco,1,5,-105.0,-115.0,...,0.252246,German Marquez,Carlos Carrasco,2022,6.16,3.73,0.747692,1.045761,0.605519,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3611,301021,2024-08-21T00:10:00+00:00,Kansas City,LA Angels,Cole Ragans,Tyler Anderson,5,9,-210.0,170.0,...,0.234300,Cole Ragans,Tyler Anderson,2024,5.92,3.40,1.296037,1.094678,0.574324,0
3612,301022,2024-08-21T00:10:00+00:00,Houston,Boston,Ronel Blanco,Nick Pivetta,5,6,-135.0,110.0,...,0.261592,Ronel Blanco,Nick Pivetta,2024,3.14,4.61,1.038168,0.998605,1.468153,0
3614,301034,2024-08-21T01:40:00+00:00,San Diego,Minnesota,Martin Perez,Bailey Ober,7,5,100.0,-120.0,...,0.251004,Martin Perez,Bailey Ober,2024,5.20,3.54,1.019784,1.056716,0.680769,1
3615,301025,2024-08-21T01:45:00+00:00,San Francisco,Chi. White Sox,Robbie Ray,Davis Martin,4,1,-250.0,200.0,...,0.219851,Robbie Ray,Davis Martin,2024,3.71,3.22,2.152542,1.106113,0.867925,1


In [31]:
df_odds_filtered.to_csv("master_df.csv", index=False)
